In [ ]:
pip install transformers rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3af3a5ccb32967c890eaba615d9ce2d3ace0945fa0da2a7f3e758ce0ff99fe97
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
pip install transformers rouge-score nltk


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import pandas as pd

# Path ke file CSV
csv_path = 'test_df.csv'

# Membaca CSV dan menghapus kolom 'Unnamed: 0'
test_df = pd.read_csv(csv_path)
test_df = test_df.drop(['Unnamed: 0'], axis=1)

# Tampilkan beberapa data untuk memastikan
test_df.head()


,original_text,extractive_summary
0,jakarta kejatuhan abdurrahman wahid dari k...,ia juga tak mendapat dukungan dari militer be...
1,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid ...
2,jakarta bank muamalat bergeming ancaman l...,bahkan bank muamalat berencana meningkatkan k...
3,jakarta secara keseluruhan lawatan preside...,jakarta secara keseluruhan lawatan preside...
4,jakarta sejumlah prajurit tni rabu 03 10...,saat memberi pengarahan widodo mengatakan se...


In [ ]:
from datasets import Dataset
test_dataset = Dataset.from_pandas(test_df)
device = 'cuda'

In [ ]:
import zipfile
import os
from transformers import RobertaTokenizer, BertForTokenClassification
import torch

# Path ke file zip
zip_path = 'final_model.zip'
extracted_model_dir = 'final_model2'

# Mengekstrak file zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_model_dir)

# Memuat tokenizer dan model BERT untuk extractive summarization
tokenizer = RobertaTokenizer.from_pretrained(extracted_model_dir)
model = BertForTokenClassification.from_pretrained(extracted_model_dir).to(device)

# Pastikan model dalam mode evaluasi
model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
import torch
from tqdm import tqdm



def generate_extractive_summaries(model, tokenizer, dataset, device):
    model.eval()
    summaries = []

    for example in tqdm(dataset, desc="Generating Extractive Summaries"):
        text = example["original_text"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().cpu().numpy())

        # Build summary by including tokens predicted as label 1
        summary_tokens = [
            token for token, pred in zip(tokens, predictions)
            if pred == 1 and token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]
        ]

        # Reconstruct text
        summary = tokenizer.convert_tokens_to_string(summary_tokens)
        summaries.append(summary.strip())

    return summaries

In [ ]:
# Assuming test_dataset is a Dataset object like train_dataset/dev_dataset
predicted_summaries = generate_extractive_summaries(model, tokenizer, test_dataset, device)

Generating Extractive Summaries: 100%|██████████| 3032/3032 [01:27<00:00, 34.51it/s]


In [ ]:
result_df = pd.DataFrame({
    "original_text": test_df["original_text"],
    "true_extractive_summary": test_df["extractive_summary"],
    "predicted_extractive_summary": predicted_summaries
})

# View in notebook
result_df.head()

,original_text,true_extractive_summary,predicted_extractive_summary
0,jakarta kejatuhan abdurrahman wahid dari k...,ia juga tak mendapat dukungan dari militer be...,jakarta kejatuhan abdurrahman wahid dari kurs...
1,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid ...,jakarta mantan presiden abdurrahman wahid men...
2,jakarta bank muamalat bergeming ancaman l...,bahkan bank muamalat berencana meningkatkan k...,jakarta bank muamalat bergeming ancaman liku...
3,jakarta secara keseluruhan lawatan preside...,jakarta secara keseluruhan lawatan preside...,jakarta secara keseluruhan lawatan presiden m...
4,jakarta sejumlah prajurit tni rabu 03 10...,saat memberi pengarahan widodo mengatakan se...,jakarta sejumlah prajurit tni u 03 10 meng...


In [ ]:
result_df.to_csv("mbert_extractive_summary_predictions.csv", index=False)
print("Saved predictions to mbert_extractive_summary_predictions.csv")

Saved predictions to mbert_extractive_summary_predictions.csv


In [ ]:
from rouge_score import rouge_scorer

# Siapkan referensi dan kandidat
references = test_df['extractive_summary'].tolist()
candidates = result_df['predicted_extractive_summary'].tolist()

# Inisialisasi Rouge Scorer dengan ROUGE-1, ROUGE-2, dan ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# List untuk menyimpan skor
rouge1_precisions = []
rouge1_recalls = []
rouge1_f1s = []

rouge2_precisions = []
rouge2_recalls = []
rouge2_f1s = []

rougeL_precisions = []
rougeL_recalls = []
rougeL_f1s = []

# Iterasi untuk setiap pasang kandidat dan referensi
for candidate, reference in zip(candidates, references):
    scores = scorer.score(reference, candidate)

    # ROUGE-1
    rouge1_precisions.append(scores['rouge1'].precision)
    rouge1_recalls.append(scores['rouge1'].recall)
    rouge1_f1s.append(scores['rouge1'].fmeasure)

    # ROUGE-2
    rouge2_precisions.append(scores['rouge2'].precision)
    rouge2_recalls.append(scores['rouge2'].recall)
    rouge2_f1s.append(scores['rouge2'].fmeasure)

    # ROUGE-L
    rougeL_precisions.append(scores['rougeL'].precision)
    rougeL_recalls.append(scores['rougeL'].recall)
    rougeL_f1s.append(scores['rougeL'].fmeasure)

# Cetak hasil agregasi
print(f"ROUGE-1 Precision: {sum(rouge1_precisions) / len(rouge1_precisions):.4f}")
print(f"ROUGE-1 Recall: {sum(rouge1_recalls) / len(rouge1_recalls):.4f}")
print(f"ROUGE-1 F1: {sum(rouge1_f1s) / len(rouge1_f1s):.4f}")

print(f"\nROUGE-2 Precision: {sum(rouge2_precisions) / len(rouge2_precisions):.4f}")
print(f"ROUGE-2 Recall: {sum(rouge2_recalls) / len(rouge2_recalls):.4f}")
print(f"ROUGE-2 F1: {sum(rouge2_f1s) / len(rouge2_f1s):.4f}")

print(f"\nROUGE-L Precision: {sum(rougeL_precisions) / len(rougeL_precisions):.4f}")
print(f"ROUGE-L Recall: {sum(rougeL_recalls) / len(rougeL_recalls):.4f}")
print(f"ROUGE-L F1: {sum(rougeL_f1s) / len(rougeL_f1s):.4f}")

ROUGE-1 Precision: 0.2065
ROUGE-1 Recall: 0.5582
ROUGE-1 F1: 0.2971

ROUGE-2 Precision: 0.1548
ROUGE-2 Recall: 0.4208
ROUGE-2 F1: 0.2230

ROUGE-L Precision: 0.1905
ROUGE-L Recall: 0.5132
ROUGE-L F1: 0.2739
